In [2]:
# --- Celda 1: Importación de librerías y archivos ---
import os
import pandas as pd
import numpy as np

# Importar funciones personalizadas
from Files.KWH.data_loader import cargar_datos_generacion
from Files.SP.StockPrice import procesar_precio_bolsa
from Penalties.penalties_v2 import hourly_payment, daily_payment
from Penalties.deviations import tolerance_range
from pydataxm.pydatasimem import ReadSIMEM

In [3]:
# --- Celda 2: Configuración de rutas y parámetros ---
# Directorios de datos
directorio_real = "Files/KWH/Real"
directorio_predicho = "Files/KWH/Predicho"

# Parámetros para el precio de bolsa
p_bolsa_id = "EC6945"
fecha_inicial = "2023-04-01"
fecha_final = "2023-04-05"



In [4]:
# --- Celda 3: Carga de datos de generación ---
from Files.KWH.data_loader import cargar_datos_generacion

# Cargar datos reales y predichos
REAL = cargar_datos_generacion(directorio_real, columna_objetivo="Pdc, W")
AHEAD = cargar_datos_generacion(directorio_predicho, columna_objetivo="Pdc, W")

# Depuración: Verificar contenido después de la carga
print("Datos reales (REAL):")
for fecha, valores in REAL.items():
    print(f"Fecha: {fecha}, Número de valores: {len(valores)}, Valores: {valores}")

print("\nDatos predichos (AHEAD):")
for fecha, valores in AHEAD.items():
    print(f"Fecha: {fecha}, Número de valores: {len(valores)}, Valores: {valores}")

# Manejo de TODAY
TODAY = AHEAD  # Si TODAY también proviene de Predicho


Procesando archivo: RP_real_04_04.csv
Procesando archivo: RP_real_02_04.csv
Procesando archivo: RP_real_05_04.csv
Procesando archivo: RP_real_03_04.csv
Procesando archivo: RP_real_01_04.csv
Procesando archivo: RP_pred_04_04.csv
Procesando archivo: RP_pred_02_04.csv
Procesando archivo: RP_pred_05_04.csv
Procesando archivo: RP_pred_01_04.csv
Procesando archivo: RP_pred_03_04.csv
Datos reales (REAL):
Fecha: 2023-04-04, Número de valores: 24, Valores: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 296.47344, 972.06395, 1559.4749399999998, 2042.6726, 2317.7989, 2504.96574, 2460.42554, 2157.5986000000003, 1816.47046, 800.0121800000001, 351.93465000000003, 37.23308, 0.16965, 0.0, 0.0, 0.0, 0.0, 0.0]
Fecha: 2023-04-02, Número de valores: 24, Valores: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 199.79232000000002, 1001.65239, 1598.84476, 2055.0255899999997, 2282.64844, 2542.13984, 2223.8197999999998, 1855.86588, 1025.22529, 1247.8233, 321.17924, 125.13242, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Fecha: 2023-04-05, Número de valores:

In [5]:
# --- Celda 4: Carga de precios de bolsa ---
simem = ReadSIMEM()
STOCK_PRICE = procesar_precio_bolsa(simem, p_bolsa_id, fecha_inicial, fecha_final)

# Mostrar precios de bolsa
print("Precios de bolsa (STOCK_PRICE):", STOCK_PRICE)



['https://www.simem.co/backend-files/api/PublicData?datasetId=EC6945&startdate=2023-04-01&enddate=2023-04-01']
['https://www.simem.co/backend-files/api/PublicData?datasetId=EC6945&startdate=2023-04-02&enddate=2023-04-02']
['https://www.simem.co/backend-files/api/PublicData?datasetId=EC6945&startdate=2023-04-03&enddate=2023-04-03']
['https://www.simem.co/backend-files/api/PublicData?datasetId=EC6945&startdate=2023-04-04&enddate=2023-04-04']
['https://www.simem.co/backend-files/api/PublicData?datasetId=EC6945&startdate=2023-04-05&enddate=2023-04-05']
Precios de bolsa (STOCK_PRICE): {'2023-04-01': [142.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 126.0, 142.0, 158.0, 158.0, 163.0, 168.0, 163.0, 158.0, 158.0, 158.0, 158.0, 166.0, 173.0, 173.0, 166.0, 158.0, 158.0], '2023-04-02': [159.0, 159.0, 149.0, 139.0, 139.0, 139.0, 139.0, 139.0, 139.0, 149.0, 159.0, 159.0, 159.0, 159.0, 159.0, 159.0, 159.0, 159.0, 159.0, 164.0, 171.0, 171.0, 164.0, 159.0], '2023-04-03': [141.0, 126.0, 126.0, 125.0, 1

In [12]:
# --- Celda 5: Cálculo de penalidades ---
def calcular_penalidades(real, ahead, today, stock_price):
    """
    Calcula penalidades horarias y diarias para un rango de fechas.
    """
    resultados = {}

    for fecha in real.keys():
        try:
            # Convertir listas a arrays de NumPy
            real_array = np.array(real[fecha], dtype=float)
            ahead_array = np.array(ahead[fecha], dtype=float)
            today_array = np.array(today[fecha], dtype=float)
            stock_price_array = np.array(stock_price[fecha], dtype=float)

            # Calcular tolerancias
            tolerance_ahead = tolerance_range(first_dispatch=True, real=np.nansum(real_array), gx=np.nansum(ahead_array))
            tolerance_today = tolerance_range(first_dispatch=True, real=np.nansum(real_array), gx=np.nansum(today_array))

            # Calcular pagos horarios
            payment_ahead = hourly_payment(tolerance=tolerance_ahead, real=real_array, gx=ahead_array, stock_price=stock_price_array)
            payment_today = hourly_payment(tolerance=tolerance_today, real=real_array, gx=today_array, stock_price=stock_price_array)

            # Calcular pago diario
            daily_payment_result = daily_payment(ahead=payment_ahead, today=payment_today)

            # Guardar resultados
            resultados[fecha] = {
                "payment_ahead": payment_ahead,
                "payment_today": payment_today,
                "daily_payment": daily_payment_result
            }

        except Exception as e:
            print(f"Error procesando la fecha {fecha}: {e}")

    return resultados

# Calcular penalidades
resultados_penalidades = calcular_penalidades(REAL, AHEAD, TODAY, STOCK_PRICE)



In [13]:
# --- Celda 6: Visualización de resultados ---
# Mostrar resultados finales
for fecha, resultado in resultados_penalidades.items():
    print(f"Fecha: {fecha}")
    print("Pago por predicción (AHEAD):", resultado["payment_ahead"])
    print("Pago por predicción (TODAY):", resultado["payment_today"])
    print("Pago diario:", resultado["daily_payment"])
    print("-----")


Fecha: 2023-04-04
Pago por predicción (AHEAD): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Pago por predicción (TODAY): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Pago diario: 0.0
-----
Fecha: 2023-04-02
Pago por predicción (AHEAD): [     0.       0.       0.       0.       0.       0.   10394.3  11648.2
  71441.  142630.2 123264.8 154284.6 158574.9 106596.1  23877.8  51393.8
  97890.4 118912.1      0.       0.       0.       0.       0.       0. ]
Pago por predicción (TODAY): [     0.       0.       0.       0.       0.       0.   10394.3  11648.2
  71441.  142630.2 123264.8 154284.6 158574.9 106596.1  23877.8  51393.8
  97890.4 118912.1      0.       0.       0.       0.       0.       0. ]
Pago diario: 1070908.2000000002
-----
Fecha: 2023-04-05
Pago por predicción (AHEAD): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Pago por predicción (TODAY): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 